# Pythreejs tutorial and example

Here is a example to use the pythreejs package to create a molecular cubic structure visualizer. 
For molecular structure, the cubic crystal system has three different phases, namely the face centre cubic (FCC), body center cubic (BCC) and simple cubic structures. 

In [4]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, Output, VBox
from traitlets import link, dlink
import math

## Initalize the vertices coordinates and face numbers

In [5]:
vertices = [
    [0, 0, 0],
    [0, 0, 1],
    [0, 1, 0],
    [0, 1, 1],
    [1, 0, 0],
    [1, 0, 1],
    [1, 1, 0],
    [1, 1, 1]
]

faces = [
    [0, 1, 3],
    [0, 3, 2],
    [0, 2, 4],
    [2, 6, 4],
    [0, 4, 1],
    [1, 4, 5],
    [2, 3, 6],
    [3, 7, 6],
    [1, 5, 3],
    [3, 5, 7],
    [4, 6, 5],
    [5, 6, 7]
]

surfaces = [
    [0, 0.5, 0.5],
    [1, 0.5, 0.5],
    [0.5, 0.5, 0],
    [0.5, 0.5, 1],
    [0.5, 0, 0.5],
    [0.5, 1, 0.5]
]

## Using the SphereGeometry to create atoms 

SphereGeometry is a class for generating sphere geometries. 

SphereGeometry(radius : Float, widthSegments : Integer, heightSegments : Integer, phiStart : Float, phiLength : Float, thetaStart : Float, thetaLength : Float)
* radius — sphere radius. Default is 1.
* widthSegments — number of horizontal segments. Minimum value is 3, and the default is 8.
* heightSegments — number of vertical segments. Minimum value is 2, and the default is 6.
* phiStart — specify horizontal starting angle. Default is 0.
* phiLength — specify horizontal sweep angle size. Default is Math.PI * 2.
* thetaStart — specify vertical starting angle. Default is 0.
* thetaLength — specify vertical sweep angle size. Default is Math.PI.

In [6]:
# balls at vertex 
balls = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi,), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in vertices]

cball = Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='gold'),
            position=[0.5, 0.5, 0.5])

sballs = [Mesh(geometry=SphereGeometry(radius=0.1, widthSegments=32, heightSegments=32,
                                     phiStart = 0, phiLength = math.pi*2,
                                     thetaStart = 0, thetaLength = math.pi), 
            material=MeshLambertMaterial(color='red', opacity = 0.7, transparent = True),
            position=i) for i in surfaces]

## Create a box geometry and obtain its edges 

BoxGeometry is a geometry class for a rectangular cuboid with a given 'width', 'height', and 'depth'. On creation, the cuboid is centred on the origin, with each edge parallel to one of the axes.

BoxGeometry(width : Float, height : Float, depth : Float, widthSegments : Integer, heightSegments : Integer, depthSegments : Integer)
* width — Width; that is, the length of the edges parallel to the X axis. Optional; defaults to 1.
* height — Height; that is, the length of the edges parallel to the Y axis. Optional; defaults to 1.
* depth — Depth; that is, the length of the edges parallel to the Z axis. Optional; defaults to 1.
* widthSegments — Number of segmented rectangular faces along the width of the sides. Optional; defaults to 1.
* heightSegments — Number of segmented rectangular faces along the height of the sides. Optional; defaults to 1.
* depthSegments — Number of segmented rectangular faces along the depth of the sides. Optional; defaults to 1.

EdgesGeometry can be used as a helper object to view the edges of a Geometry object. 

EdgesGeometry( geometry : Geometry, thresholdAngle : Integer )
* geometry — Any geometry object.
* thresholdAngle — An edge is only rendered if the angle (in degrees) between the face normals of the adjoining faces exceeds this value. default = 1 degree.

In [7]:
box = BoxGeometry(1,1,1)
edgesGemoetry = EdgesGeometry(box);

## Create mesh for surfaces and line segments for edges

Mesh class representing triangular polygon mesh based objects. Also serves as a base for other classes.

Mesh( geometry : Geometry, material : Material )
* geometry — (optional) an instance of Geometry or BufferGeometry. Default is a new BufferGeometry.
* material — (optional) a single or an array of Material. Default is a new MeshBasicMaterial

LineSegments creates a series of lines drawn between pairs of vertices.

LineSegments( geometry : Geometry, material : Material )
* geometry — Pair(s) of vertices representing each line segment(s).
* material — Material for the line. Default is LineBasicMaterial.

In [8]:
# Create a mesh. Note that the material need to be told to use the vertex colors.
myobjectCube = Mesh(
    geometry=box,
    material= MeshPhongMaterial(color = 'blue', opacity = 0.4, transparent = True),
    position=[0.5, 0.5, 0.5],   # Center the cube
)


edges = LineSegments(
    geometry = edgesGemoetry,
    material = LineBasicMaterial(color = 'black', linewidth = 3),
    position = [0.5, 0.5, 0.5]
)

## Create a camera

Camera that uses perspective projection. The perspective projection mode is designed to mimic the way the human eye sees. It is the most common projection mode used for rendering a 3D scene. 

In [9]:
# Set up a scene and render it:
cCube = PerspectiveCamera(position=[3, 3, 3], fov=20,
                      children=[DirectionalLight(color='#ffffff', position=[-3, 5, 1], intensity=0.5)])

## Create a scene

Scenes allow you to set up what and where is to be rendered by three.js. This is where you place objects, lights and cameras.

In [10]:
sceneCube = Scene(children= balls + sballs + [myobjectCube, cCube, cball, edges, AmbientLight(color='#dddddd')])

## Render the objects 

In [11]:
rendererCube = Renderer(camera=cCube, background='black', background_opacity=1,
                        scene=sceneCube, controls=[OrbitControls(controlling=cCube)], 
                        width=500, height=500)

In [12]:
from ipywidgets import HBox, VBox, Button, RadioButtons, FloatSlider 

def on_value_change(change):
    if structureSelection.value == 'Simple Cubic':
        cball.visible = False
        for i in sballs:
            i.visible = False
    elif structureSelection.value == 'BCC':
        cball.visible = True
        for i in sballs:
            i.visible = False
    else:
        cball.visible = False
        for i in sballs:
            i.visible = True

def on_radius_change(change):
    for i in balls + sballs:
        i.scale = [10*radius.value, 10*radius.value, 10*radius.value]
        
    cball.scale = [10*radius.value, 10*radius.value, 10*radius.value]
    
    
structureSelection = RadioButtons(
        options = ['FCC', 'BCC', 'Simple Cubic'],
        description = 'Structures',
        disabled = False)

structureSelection.observe(on_value_change, names='value')

radius = FloatSlider(
    value=0.0,
    min=0.0,
    max=0.5,
    step=0.1,
    description='Radius (a):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

radius.observe(on_radius_change, names = 'value')

display(VBox([structureSelection, rendererCube, radius]))

This work has been done with the support of the EPFL Open Science found [OSSCAR](http://www.osscar.org).

<img src="http://www.osscar.org/wp-content/uploads/2019/03/OSSCAR-logo.png" style="height:40px; width: 200px"/>